In [5]:
##Generate dataset

import cv2
import numpy as np

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml") #import haarcascade classifier
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # convert to gray scale image
        faces = face_classifier.detectMultiScale(gray, 1.3, 5) # scaling image 
        # scaling factor = 1.3
        # minimum neighbor = 5
        
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w] #Cropping image
        return cropped_face
    
    cap = cv2.VideoCapture(0) #capturing image from laptop camera
    id =2            #user id
    img_id = 0       #image ids
    
    while True:
        ret, frame = cap.read()  # read captured image
        if face_cropped(frame) is not None: # if image frames are not none increase image id by 1
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200)) # coordinates ot rows and coloumns
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) # convert it to gray scale
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg" #store images to this path
             #file_name_path = './Images/' + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face) # store image to file_name_path
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2) #putting image text
            
            #(50,50) - is the orgin point from where text is to be written
            #font scale=1
            #thickness=2
            
            
            cv2.imshow("Cropped face", face) # show cropped image
            
        if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")
generate_dataset()

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\danny\AppData\Local\Temp/ipykernel_15448/1314559618.py:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [2]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy

def train_classifier(data_dir): #data directory 
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)] # join all the images in data folder to path
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image) # convert image to gray scale
        imageNp = np.array(img, 'uint8')   # convert image to array
        id = int(os.path.split(image)[1].split(".")[1])  #spliitng images by '.' to get id number from image 
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids) # training faces n ids
    clf.write("classifier.xml") # saving trained faces and ids to xml
train_classifier("data")

In [6]:
import cv2
import numpy as np
from PIL import Image
import os

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf): 
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   # Gray scale image
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors) #MultiScale detects objects of different sizes 
     # in the input image 
    #scalefactor (how much the image size will be reduced at each image scale)
    #  minNeighbors (how many neighbors each rectangle should have)
    
    coords= []
    
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 ) # draws rectangle over live face
        
        id, pred = clf.predict(gray_img[y:y+h,x:x+w]) # predicts id and pred to calculate confidence
        confidence = int(100*(1-pred/300))
        if confidence>74:
            if id==1:
                cv2.putText(img, "Barah", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id ==2:
                cv2.putText(img, "Danny", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id ==3:
                cv2.putText(img, "Catherine", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id ==4:
                cv2.putText(img, "Bindu", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
            
        coords =[x,y,w,h]    
    
    return coords


def recognize(img, clf,faceCascade):
    coords=draw_boundary(img,faceCascade,1.1,10, (0,255,0), "Face", clf) #draws rectangle boundary over face
    return img


# loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

clf = cv2.face.LBPHFaceRecognizer_create()  # trained Local Binary Pattern histogram classifier
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()  # read images from video
    img = recognize(img, clf,faceCascade) # will detect the face of a person whose images are already trained and draw the rectangles and display the name.

    cv2.imshow("face Detection", img)  #shows image
    
    if cv2.waitKey(1)==13:
        break
video_capture.release()
cv2.destroyAllWindows()